In [ ]:
# 1. Uninstall the broken versions
!pip uninstall -y pillow ultralytics

# 2. Install the stable "Golden Combo"
!pip install pillow==10.4.0 ultralytics roboflow --no-cache-dir -q

# 3. Force a kernel restart to apply changes
import os
os.kill(os.getpid(), 9)

Found existing installation: pillow 11.3.0
Uninstalling pillow-11.3.0:
  Successfully uninstalled pillow-11.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 374.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 232.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 349.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 199.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 341.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 339.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 175.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-adk 1.21.0 requires google-cloud-bigquery-storage>=2.0.0, which is not installed.
dopamine-rl 4.1.2 requires gym<=0.25.2, but you have gym 0.26.2

In [1]:
import PIL
from ultralytics import YOLO
from roboflow import Roboflow
import os
import yaml

# 1. Verify the fix
print(f"✅ Pillow version: {PIL.__version__}") 
# If this says 10.4.0, we are golden.

# 2. Download your specific Obstacle Dataset
rf = Roboflow(api_key="6h0gslrCLiw8YLMToXU9")
project = rf.workspace("h-ylfot").project("obstacle-xy6vw")
version = project.version(1)
dataset = version.download("yolov8")

# 3. Setup paths for the training
dataset_path = dataset.location
yaml_path = os.path.join(dataset_path, "data.yaml")
print(f"✅ Dataset downloaded to: {dataset_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
✅ Pillow version: 10.4.0
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to obstacle-1 in yolov8:: 100%|██████████| 6691/6691 [00:00<00:00, 7018.26it/s]

✅ Dataset downloaded to: /kaggle/working/obstacle-1


In [2]:
# 1. Fix the YAML paths for Colab environment
with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

# Point YOLO exactly to the image folders
config['train'] = os.path.join(dataset_path, 'train', 'images')
config['val'] = os.path.join(dataset_path, 'valid', 'images')

with open(yaml_path, 'w') as f:
    yaml.dump(config, f)

# 2. Start Training the Obstacle Detector
# We use yolov8n.pt (Nano) because it's the fastest for a Rover's hardware
model = YOLO('yolov8n.pt') 

model.train(
    data=yaml_path,
    epochs=25,
    imgsz=640,
    device=0,           # Uses the NVIDIA GPU
    project='Rover_Systems',
    name='Obstacle_Detector_Final'
)

Ultralytics 8.4.17 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/obstacle-1/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=Obstacle_Detector_Final, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mas

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a008dafe330>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042, 